<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/Crime_Types_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 8.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=5ae3489174e871ec9b608e6804c67618d69e756b7fdd27f90fe59fd7206acc65
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 19 20:52 kaggle.json


In [3]:
!kaggle competitions download -c crime-types

  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 11.2MB/s]


In [4]:
!unzip crime-types.zip

Archive:  crime-types.zip
  inflating: submission_format.csv   
  inflating: test_data.csv           
  inflating: train_data.csv          


In [25]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import torch
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 10000
batch_size = 100

# Data load
train_data = pd.read_csv('train_data.csv', header=None, skiprows=1, usecols=range(0, 13))
test_data = pd.read_csv('test_data.csv', header=None, skiprows=1, usecols=range(0, 12))

# Data 파싱
x_train_data = train_data.loc[:, 1:13]
y_train_data = train_data.loc[:, 0]


# 파싱한 Data를 numpy의 array로 변환
x_train_data = np.array(x_train_data)
y_train_data = np.array(y_train_data)

test_data = np.array(test_data)

# 변환한 numpy의 array를 Tensor로 변환
x_train_data = torch.FloatTensor(x_train_data)
y_train_data = torch.LongTensor(y_train_data)

test_data = torch.FloatTensor(test_data)

# data_loader에 이용할 하나의 train Dataset으로 변환
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)

# data_loader 설정
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

# 모델 설계
linear1 = torch.nn.Linear(12, 64, bias=True)
linear2 = torch.nn.Linear(64, 64, bias=True)
linear3 = torch.nn.Linear(64,  7, bias=True)
relu = torch.nn.ReLU()

torch.nn.init.kaiming_uniform(linear1.weight)
torch.nn.init.kaiming_uniform(linear2.weight)
torch.nn.init.kaiming_uniform(linear3.weight)

#dropout = torch.nn.Dropout(p=0.2)
model = torch.nn.Sequential(linear1, relu,
                            linear2, relu,
                            linear3).to(device)

loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습
total_batch = len(data_loader)

for epoch in range(training_epochs) :
    avg_cost = 0

    for X, Y in data_loader :

        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch : {:4d}'.format(epoch+1), 'Cost : {:.9f}'.format(avg_cost))

print('Learning Finishied')

# 모델 평가
with torch.no_grad() :
    model.eval()
    test_data = test_data.to(device)

    prediction = model(test_data)
    prediction = torch.argmax(prediction, 1)
    prediction = prediction.cpu().numpy().reshape(-1, 1)

submit = pd.read_csv('submission_format.csv')

for i in range(len(prediction)) :
    submit['Lable'][i] = prediction[i].item()

submit.to_csv('result.csv', index=False, header=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch : 5003 Cost : 0.439565867
Epoch : 5004 Cost : 0.436255753
Epoch : 5005 Cost : 0.450522065
Epoch : 5006 Cost : 0.455401003
Epoch : 5007 Cost : 0.434825540
Epoch : 5008 Cost : 0.442083687
Epoch : 5009 Cost : 0.458786249
Epoch : 5010 Cost : 0.444767118
Epoch : 5011 Cost : 0.449024945
Epoch : 5012 Cost : 0.457979530
Epoch : 5013 Cost : 0.460089684
Epoch : 5014 Cost : 0.457340777
Epoch : 5015 Cost : 0.488621175
Epoch : 5016 Cost : 0.473630637
Epoch : 5017 Cost : 0.427630395
Epoch : 5018 Cost : 0.469918668
Epoch : 5019 Cost : 0.462779492
Epoch : 5020 Cost : 0.434866726
Epoch : 5021 Cost : 0.477186233
Epoch : 5022 Cost : 0.446528137
Epoch : 5023 Cost : 0.461488366
Epoch : 5024 Cost : 0.454383194
Epoch : 5025 Cost : 0.426012933
Epoch : 5026 Cost : 0.436154217
Epoch : 5027 Cost : 0.428793371
Epoch : 5028 Cost : 0.422768116
Epoch : 5029 Cost : 0.432327807
Epoch : 5030 Cost : 0.439664274
Epoch : 5031 Cost : 0.435027152
Epoch : 5032 Cost : 0.428822428
Epoc

In [26]:

!kaggle competitions submit -c crime-types -f result.csv -m "14010974_이기택"

100% 237/237 [00:04<00:00, 51.0B/s]
Successfully submitted to Classification of crime types